In [1]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util
import glob
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from transformers import AutoTokenizer, AutoModel

In [2]:
# classification
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

# Clustering
import sklearn.cluster

classifiers = [
    GaussianProcessClassifier(kernel=1.0 * RBF(1.0),random_state=0,max_iter_predict=200)
    # MLPClassifier(alpha=1, max_iter=2000),
    ]

clusters=[ KMeans(n_clusters=2, random_state=0, n_init="auto"),
        #    sklearn.cluster.DBSCAN(eps=1.25)
        ]


In [8]:
script_AD=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\AD"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df1 = pd.read_csv(f)
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
    script_AD.append(df1)


df_AD=pd.concat(script_AD,axis=0,ignore_index=True)
label_AD=0
df_AD['label']=label_AD


In [9]:
script_HC=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\Data\HC"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df2=pd.read_csv(f)
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    script_HC.append(df2)


df_HC=pd.concat(script_HC,axis=0,ignore_index=True)
label_HC=1
df_HC['label']=label_HC


In [10]:
df=pd.concat([df_AD,df_HC], ignore_index=True)
df

,sentence,label
0,mhm .,0
1,alright .,0
2,there's a young boy that's getting a cookie jar .,0
3,and he's in bad shape because the thing is fal...,0
4,and in the picture the mother is washin(g) dis...,0
...,...,...
2612,and the stool is tipping over .,1
2613,and he's sort_of put down the plates .,1
2614,and she's reaching up to get it but I don't se...,1
2615,yeah that's it .,1


In [ ]:
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences=df['sentence']

In [ ]:
input_ids = [torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)) for sentence in sentences]

# Pad the sequences to have the same length
max_len = max([len(input_id) for input_id in input_ids])
padded_input_ids = [torch.cat((input_id, torch.zeros((max_len - len(input_id)), dtype=torch.long))) for input_id in input_ids]
padded_input_ids = torch.stack(padded_input_ids).to(torch.int64)

In [ ]:
print(max_len)
print(padded_input_ids.shape)

47
torch.Size([2617, 47])


In [ ]:
with torch.no_grad():
    output = model(padded_input_ids)
    last_hidden_states = output[2] #input for next part like forward pass


In [ ]:
len(last_hidden_states[0])

2617

In [ ]:
'''
shape of single word encoder: [2617,47,768], each word: [1,768],
each sentence: [1,47,768] shape.
'''
# sum of last for encoder layers' encoding
word_embeddings= torch.stack(last_hidden_states[:5]).sum(0)
print(word_embeddings.shape)

# mean of each word embedding, final sentence embedding length:[1,768]
sentences_embedding = word_embeddings.mean(dim=1) 
print(sentences_embedding.shape) # single sentence of length 768

torch.Size([2617, 47, 768])
torch.Size([2617, 768])


In [ ]:
x=sentences_embedding
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# model(# GaussianProcessClassifier result is 68%(at layer:3))
model = GaussianProcessClassifier(kernel=1.0 * RBF(1.0),random_state=0,max_iter_predict=200)
model.fit(x_train,y_train)


GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=1),
                          max_iter_predict=200, random_state=0)

In [ ]:
y_pred=model.predict(x_test)
print(f"print score for :{metrics.accuracy_score(y_test,y_pred)}")

print score for :0.6730279898218829
